In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import datetime
import gc

## Creating Graphs

In [ ]:
ALL_PHIS_DATA = "/content/drive/MyDrive/data/all_phis_trans.csv"
phis_df = pd.read_csv(ALL_PHIS_DATA)
phis_df.head()

,Unnamed: 0,Unnamed: 0.1,from,to,amount,timestamp,fromIsPhi,toIsPhi
0,0,77056,0x40949225c4a1745a9946f6aaf763241c082cb9ac,0x336a3f0c4d40b1e39cee72e2e59330daf484848c,0.2,1524902918,1,1
1,1,77057,0x40949225c4a1745a9946f6aaf763241c082cb9ac,0x336a3f0c4d40b1e39cee72e2e59330daf484848c,0.2,1524902918,1,1
2,2,77058,0x40949225c4a1745a9946f6aaf763241c082cb9ac,0x336a3f0c4d40b1e39cee72e2e59330daf484848c,0.2,1524902918,1,1
3,3,77059,0x40949225c4a1745a9946f6aaf763241c082cb9ac,0x84cd26c3ea544f9cbb1d2df5c195fcbb8975ceaf,0.1,1523193681,1,0
4,4,77060,0x40949225c4a1745a9946f6aaf763241c082cb9ac,0x84cd26c3ea544f9cbb1d2df5c195fcbb8975ceaf,0.1,1523193681,1,0


In [ ]:
phis_df.shape, phis_df.isnull().values.any()

((59999, 8), False)

In [ ]:
phis_df["date"] = pd.to_datetime(phis_df["timestamp"].astype(str),unit='s')
phis_df["date"] = phis_df["date"].dt.date
DATE_RANGE = (phis_df["date"].min(), phis_df["date"].max())
DATE_RANGE

(datetime.date(2016, 11, 30), datetime.date(2019, 1, 17))

In [ ]:
phis_df["from"] = phis_df["from"].apply(int, base=16)
phis_df["to"] = phis_df["to"].apply(int, base=16)

In [ ]:
# a size 2 window of consecutive transactions
def divide_in_time_windows(df, window=2, start_index_from=0):
  dates = df["date"].unique()
  dates.sort()
  enumerated_dates = np.array(list(enumerate(dates)))
  i = 1
  while i <= enumerated_dates.shape[0]-window:
    j = 1
    label = True
    enumerated_dates[i][0] = enumerated_dates[i-1][0]+1
    while j < window:
      if (enumerated_dates[i+j][1]-enumerated_dates[i+j-1][1]).days > 1:
        label = False
      j+=1
    if label:
      enumerated_dates[i:i+window, 0] = enumerated_dates[i][0]
      i += window
    else :
      i += 1
  if window == 3:
    enumerated_dates[-1, 0] = enumerated_dates[-2, 0] + 1
  if window == 4:
    enumerated_dates[-1, 0] = enumerated_dates[-3, 0] + 1
    enumerated_dates[-2, 0] = enumerated_dates[-3, 0] + 1
  dates_lookup = dict((d, indx+start_index_from) for indx, d in enumerated_dates)
  [print(x) for x in enumerated_dates]
  df['idx'] = df['date'].map(dates_lookup)
  graphs = df.set_index(['idx','date']).sort_index()
  return graphs

In [ ]:
pd.options.display.max_rows = 4000

In [ ]:
def combine_graphs(graphs, phis_graphs):
  all_graphs = pd.concat([graphs, phis_graphs]).sort_index().reset_index()
  return all_graphs

In [ ]:
# get a specific window and flatten dataframe
def get_id(graphs, id):
  filt = graphs.index.get_level_values('idx') == id
  new_graph = graphs[filt].copy()
  return new_graph.reset_index()

# convert this edgelist dataframe to graph and set node attributes
def convert_df_graph(df):
  df["date"] = df["date"].astype(str)
  G = nx.convert_matrix.from_pandas_edgelist(df, "from", "to", edge_attr=True,create_using=nx.DiGraph())
  node_attrs = {}
  for node in G.nodes():
    node_attrs[node] = {"addr": node}

  for src, dest, data in G.edges(data=True):
    node_attrs[src]["isPhi"] = bool(data['fromIsPhi'])
    node_attrs[dest]["isPhi"] = bool(data['toIsPhi'])

  nx.classes.function.set_node_attributes(G, node_attrs)
  return G

In [ ]:
DATA = "/content/drive/MyDrive/data/ethereum_data.csv"
df = pd.read_csv(DATA)
df.head()

,Unnamed: 0,from,to,amount,timestamp,fromIsPhi,toIsPhi
0,0,0x1f1e784a61a8ca0a90250bcd2170696655b28a21,0x1266f8b9e4dffc9e2f719bf51713f7e714516861,2.344623,1.526454e+09,0,0
1,1,0x1f1e784a61a8ca0a90250bcd2170696655b28a21,0x806ceb189d36700a97f4e7ecd4fb6c95f2c5a3de,0.070000,1.504462e+09,0,0
2,2,0x1f1e784a61a8ca0a90250bcd2170696655b28a21,0x806ceb189d36700a97f4e7ecd4fb6c95f2c5a3de,0.070000,1.504462e+09,0,0
3,3,0x1f1e784a61a8ca0a90250bcd2170696655b28a21,0x3ec4688db6bf8464b0bef30ec2ca7afc7fa7863c,5.068543,1.507806e+09,0,0
4,4,0x1f1e784a61a8ca0a90250bcd2170696655b28a21,0x3ec4688db6bf8464b0bef30ec2ca7afc7fa7863c,5.068543,1.507806e+09,0,0


In [ ]:
df.shape, df.isnull().values.any()

((13551303, 7), False)

In [ ]:
df["date"] = pd.to_datetime(df["timestamp"].astype(str),unit='s')
df["date"] = df["date"].dt.date
# remove transactions that are made outside the time frame where phishing transactions were made.
outside_range_index = df[(df["date"]<DATE_RANGE[0]) | (df["date"]>DATE_RANGE[1])].index
df.drop(outside_range_index, inplace=True)
phis_indices = df[(df['toIsPhi']==1) | (df['fromIsPhi']==1)].index
df.drop(phis_indices, inplace=True)
df["from"] = df["from"].apply(int, base=16)
df["to"] = df["to"].apply(int, base=16)

In [ ]:
df.shape

(13048067, 8)

In [ ]:
df["isPhi"] = df["fromIsPhi"] | df["toIsPhi"]
phis_df["isPhi"] = phis_df["fromIsPhi"] | phis_df["toIsPhi"]

In [ ]:
graphs = divide_in_time_windows(df, window=4)

[0 datetime.date(2016, 11, 30)]
[1 datetime.date(2016, 12, 1)]
[1 datetime.date(2016, 12, 2)]
[1 datetime.date(2016, 12, 3)]
[1 datetime.date(2016, 12, 4)]
[2 datetime.date(2016, 12, 5)]
[2 datetime.date(2016, 12, 6)]
[2 datetime.date(2016, 12, 7)]
[2 datetime.date(2016, 12, 8)]
[3 datetime.date(2016, 12, 9)]
[3 datetime.date(2016, 12, 10)]
[3 datetime.date(2016, 12, 11)]
[3 datetime.date(2016, 12, 12)]
[4 datetime.date(2016, 12, 13)]
[4 datetime.date(2016, 12, 14)]
[4 datetime.date(2016, 12, 15)]
[4 datetime.date(2016, 12, 16)]
[5 datetime.date(2016, 12, 17)]
[5 datetime.date(2016, 12, 18)]
[5 datetime.date(2016, 12, 19)]
[5 datetime.date(2016, 12, 20)]
[6 datetime.date(2016, 12, 21)]
[6 datetime.date(2016, 12, 22)]
[6 datetime.date(2016, 12, 23)]
[6 datetime.date(2016, 12, 24)]
[7 datetime.date(2016, 12, 25)]
[7 datetime.date(2016, 12, 26)]
[7 datetime.date(2016, 12, 27)]
[7 datetime.date(2016, 12, 28)]
[8 datetime.date(2016, 12, 29)]
[8 datetime.date(2016, 12, 30)]
[8 datetime.date(

In [ ]:
phis_graphs = divide_in_time_windows(phis_df,window=4, start_index_from=graphs.index.get_level_values('idx').max()+1)


[0 datetime.date(2016, 11, 30)]
[1 datetime.date(2017, 3, 4)]
[2 datetime.date(2017, 3, 5)]
[3 datetime.date(2017, 5, 27)]
[4 datetime.date(2017, 6, 4)]
[5 datetime.date(2017, 6, 6)]
[6 datetime.date(2017, 6, 7)]
[7 datetime.date(2017, 6, 14)]
[7 datetime.date(2017, 6, 15)]
[7 datetime.date(2017, 6, 16)]
[7 datetime.date(2017, 6, 17)]
[8 datetime.date(2017, 6, 18)]
[8 datetime.date(2017, 6, 19)]
[8 datetime.date(2017, 6, 20)]
[8 datetime.date(2017, 6, 21)]
[9 datetime.date(2017, 6, 22)]
[9 datetime.date(2017, 6, 23)]
[9 datetime.date(2017, 6, 24)]
[9 datetime.date(2017, 6, 25)]
[10 datetime.date(2017, 6, 26)]
[10 datetime.date(2017, 6, 27)]
[10 datetime.date(2017, 6, 28)]
[10 datetime.date(2017, 6, 29)]
[11 datetime.date(2017, 6, 30)]
[11 datetime.date(2017, 7, 1)]
[11 datetime.date(2017, 7, 2)]
[11 datetime.date(2017, 7, 3)]
[12 datetime.date(2017, 7, 4)]
[12 datetime.date(2017, 7, 5)]
[12 datetime.date(2017, 7, 6)]
[12 datetime.date(2017, 7, 7)]
[13 datetime.date(2017, 7, 8)]
[13 dat

In [ ]:
all_graphs = combine_graphs(graphs, phis_graphs)

In [ ]:
del df
del phis_df
del graphs
del phis_graphs

In [ ]:
all_graphs['idx'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [ ]:
all_graphs[all_graphs.idx == 100]

,idx,date,Unnamed: 0,from,to,amount,timestamp,fromIsPhi,toIsPhi,isPhi,Unnamed: 0.1
5148998,100,2018-01-01,1665,1033618310653199987383472215097185760827692870773,1050670650477272892465283503093290026457891954182,0.004000,1.514798e+09,0,0,0,NaN
5148999,100,2018-01-01,1666,1033618310653199987383472215097185760827692870773,397569076913962412146951693727562543262325959432,0.050000,1.514833e+09,0,0,0,NaN
5149000,100,2018-01-01,12180,492293783146356869670525059291990467955676821777,276474070519076501484329038292647496012550751838,0.025180,1.514792e+09,0,0,0,NaN
5149001,100,2018-01-01,12181,492293783146356869670525059291990467955676821777,276474070519076501484329038292647496012550751838,0.025180,1.514792e+09,0,0,0,NaN
5149002,100,2018-01-01,12182,492293783146356869670525059291990467955676821777,276474070519076501484329038292647496012550751838,0.025180,1.514792e+09,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5267552,100,2018-01-04,13499252,628839617637661431357097493460327659252253447652,779484237962539308017777060108035793541394695034,0.695259,1.515045e+09,0,0,0,NaN
5267553,100,2018-01-04,13514536,832634744962534730927423627800495590623790237251,1292486350761514995442236621832964271905274491408,0.000000,1.515042e+09,0,0,0,NaN
5267554,100,2018-01-04,13514537,775715667558944944114095589557755989764493713069,1292486350761514995442236621832964271905274491408,0.000000,1.515044e+09,0,0,0,NaN
5267555,100,2018-01-04,13514538,807127800132241013510267570436263052056357689999,1292486350761514995442236621832964271905274491408,0.000000,1.515103e+09,0,0,0,NaN


In [ ]:
GRAPH_PATHS = "/content/drive/MyDrive/data/processed_graphs_win4.csv"

In [ ]:
all_graphs.to_csv(GRAPH_PATHS,index=False)

## Shuffling Graphs and Creating Graph Labels

In [ ]:
graphs = pd.read_csv(GRAPH_PATHS)

In [ ]:
repeating_labels = graphs.filter(['idx', 'isPhi'], axis=1)

In [ ]:
labels = repeating_labels.groupby('idx').mean()

In [ ]:
LABELS_PATHS = "/content/drive/MyDrive/data/processed_graphs_labels_win4.csv"

In [ ]:
labels

,isPhi
idx,
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0


In [ ]:
labels.to_csv(LABELS_PATHS)